In [83]:
# import dependencies
# import pandas as pd
# import numpy as np
import random
import json


In [84]:
# select desired state
state = 'missouri'

# access json file with data to be sonified for avaliable states
stateData = json.load(open('../resources/stateData.json'))

# display dictionary
stateData


{'tennessee': {'white': {'population': 5200531,
   'inverted_ccvi': 45.91,
   'chance_of_infection': 15.44,
   'chance_of_death': 1.47,
   'generated_cases': 0,
   'generated_deaths': 0},
  'black': {'population': 1120221,
   'inverted_ccvi': 28.16,
   'chance_of_infection': 15.65,
   'chance_of_death': 1.68,
   'generated_cases': 0,
   'generated_deaths': 0},
  'native': {'population': 18151,
   'inverted_ccvi': 40.66,
   'chance_of_infection': 9.31,
   'chance_of_death': 1.12,
   'generated_cases': 0,
   'generated_deaths': 0},
  'asian': {'population': 117348,
   'inverted_ccvi': 49.7,
   'chance_of_infection': 9.41,
   'chance_of_death': 0.79,
   'generated_cases': 0,
   'generated_deaths': 0},
  'pacific': {'population': 3735,
   'inverted_ccvi': 37.37,
   'chance_of_infection': 24.52,
   'chance_of_death': 1.03,
   'generated_cases': 0,
   'generated_deaths': 0},
  'hispanic': {'population': 363753,
   'inverted_ccvi': 32.45,
   'chance_of_infection': 25.67,
   'chance_of_death':

In [85]:
# list to hold probabilities for which demographic a new case will belong to
infectProbs = []
# list to hold probability that an infection will result in death for each demographic
deathProbs = []

# populate lists above
for data in stateData[state]:
    infectProbs.append(stateData[state][data]['chance_of_infection'])
    deathProbs.append(stateData[state][data]['chance_of_death'])


In [86]:
# weighted probabilities for #people the current case can spread infection to
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
spreadProbs = [0,0,0,12]
# symptomatic cases that have occured
sympCount = 0
# deaths that have occured
deathCount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# transposition in semitones
trans = 0
# cases remaining (one case to start it off)
more = [0]
# which case
case = 0
# array of midi notes
midiNotes = [48, 51, 55, 58, 62, 63, 65, 67, 68, 68, 74, 77, 78, 80]
# dictionary with all data
weathering = {
    'time': [],
    'demo': [],
    'note': [],
    'death': [],
    'n': [],
}


In [87]:
# function to determine how many cases correspond to a particular threshold
def nthThresh(n,t):
    return 100 * n + t

# function to adjust spreadProbs weights and calculate r0 for corresponding threshold
def thisThresh():
    if n == 1:
        spreadProbs[3] -= 1
        spreadProbs[2] += 1
    elif n == 2:
        spreadProbs[3] -= 1
        spreadProbs[1] += 1
    elif n == 13:
        spreadProbs[2] -= 1
        spreadProbs[0] += 1
    elif n == 14:
        spreadProbs[1] -= 1
        spreadProbs[0] += 1
    elif n != 0 and n < 13:
        spreadProbs[3] -= 1
        spreadProbs[0] += 1
    r0 = (spreadProbs[1] + 2*spreadProbs[2] + 3*spreadProbs[3]) / sum(spreadProbs)

# function to select rescaled incubation period as a delay in milliseconds.
def incubate(n):
    if n == 1:  
        sub = 3
        return random.randint(0, sub*40)*2000/sub
    elif n == 2:
        sub = random.choice([3,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 3:
        sub = random.choice([6,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 4:
        sub = random.choice([6,8])
        return random.randint(0, sub*40)*2000/sub
    elif n == 5:
        sub = random.choice([6,8,5])
        return random.randint(0, sub*40)*2000/sub
    else:
        sub = random.choice([6,8,5,7])
        return random.randint(0, sub*40)*2000/sub

# function to select a value based on a probability distribution array
def select(probsArray):
    return random.choices(list(range(len(probsArray))),weights=probsArray)[0]

# function to decide between one and zero based on probability that 1 will be chosen
def decide(prob):
    return random.choices([0,1], weights=[100-prob, prob])[0]

# function to generate midi notes
def midi(t):
    note = random.choice(midiNotes)
    return note + t

In [88]:
while len(more) > 0:

    # decide how many people the current case infects and add them to more
    for i in range(select(spreadProbs)):
        more.append(case)

#     # decide if asymptomatic or not
#     if decide(40) == 0:

    # add current case to total # of cases
    # (only sympotmatic cases adjust threshold)
    sympCount += 1

    # if case count is at or over threshold number
    if sympCount > nthThresh(n, thresh):
        # calculate new threshold
        thresh = nthThresh(n, thresh)
        # adjust spreadProbs weights and calculate r0 for corresponding threshold
        thisThresh()
        # add one to threshold n value
        n += 1

    # select which demographic the current case belongs to
    demo = select(infectProbs)
    weathering['demo'].append(demo)

    # select which note
    weathering['note'].append(midi(0))

    # select incubation period (delay time)
    delay = incubate(n)

    # calculate time from beginning that symptoms occur for current case
    if sympCount == 1:
        time = delay
    else:
        time = delay + weathering['time'][more[0]]
    weathering['time'].append(time)

    # decide/record if case results in death or not
    death = decide(deathProbs[demo])
    weathering['death'].append(death)
    
    weathering['n'].append(n)

    case += 1

    # remove current case from how many more cases left
    more.pop(0)


In [89]:
result = list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death'], weathering['n']))

# result = sorted(result, reverse=False)

len(result)


11389

In [90]:
for i, r in enumerate(result):
#     print(f'{r}: {result[i][4]}')
    print(f'{i}: {r[4]}')

0: 1
1: 1
2: 1
3: 1
4: 1
5: 1
6: 1
7: 1
8: 1
9: 1
10: 1
11: 1
12: 1
13: 1
14: 1
15: 1
16: 1
17: 1
18: 1
19: 1
20: 1
21: 1
22: 1
23: 1
24: 1
25: 1
26: 1
27: 1
28: 1
29: 1
30: 1
31: 1
32: 1
33: 1
34: 1
35: 1
36: 1
37: 1
38: 1
39: 1
40: 1
41: 1
42: 1
43: 1
44: 1
45: 1
46: 1
47: 1
48: 1
49: 1
50: 1
51: 1
52: 1
53: 1
54: 1
55: 1
56: 1
57: 1
58: 1
59: 1
60: 1
61: 1
62: 1
63: 1
64: 1
65: 1
66: 1
67: 1
68: 1
69: 1
70: 1
71: 1
72: 1
73: 1
74: 1
75: 1
76: 1
77: 1
78: 1
79: 1
80: 1
81: 1
82: 1
83: 1
84: 1
85: 1
86: 1
87: 1
88: 1
89: 1
90: 1
91: 1
92: 1
93: 1
94: 1
95: 1
96: 1
97: 1
98: 1
99: 1
100: 2
101: 2
102: 2
103: 2
104: 2
105: 2
106: 2
107: 2
108: 2
109: 2
110: 2
111: 2
112: 2
113: 2
114: 2
115: 2
116: 2
117: 2
118: 2
119: 2
120: 2
121: 2
122: 2
123: 2
124: 2
125: 2
126: 2
127: 2
128: 2
129: 2
130: 2
131: 2
132: 2
133: 2
134: 2
135: 2
136: 2
137: 2
138: 2
139: 2
140: 2
141: 2
142: 2
143: 2
144: 2
145: 2
146: 2
147: 2
148: 2
149: 2
150: 2
151: 2
152: 2
153: 2
154: 2
155: 2
156: 2
157: 2
158:

1886: 6
1887: 6
1888: 6
1889: 6
1890: 6
1891: 6
1892: 6
1893: 6
1894: 6
1895: 6
1896: 6
1897: 6
1898: 6
1899: 6
1900: 6
1901: 6
1902: 6
1903: 6
1904: 6
1905: 6
1906: 6
1907: 6
1908: 6
1909: 6
1910: 6
1911: 6
1912: 6
1913: 6
1914: 6
1915: 6
1916: 6
1917: 6
1918: 6
1919: 6
1920: 6
1921: 6
1922: 6
1923: 6
1924: 6
1925: 6
1926: 6
1927: 6
1928: 6
1929: 6
1930: 6
1931: 6
1932: 6
1933: 6
1934: 6
1935: 6
1936: 6
1937: 6
1938: 6
1939: 6
1940: 6
1941: 6
1942: 6
1943: 6
1944: 6
1945: 6
1946: 6
1947: 6
1948: 6
1949: 6
1950: 6
1951: 6
1952: 6
1953: 6
1954: 6
1955: 6
1956: 6
1957: 6
1958: 6
1959: 6
1960: 6
1961: 6
1962: 6
1963: 6
1964: 6
1965: 6
1966: 6
1967: 6
1968: 6
1969: 6
1970: 6
1971: 6
1972: 6
1973: 6
1974: 6
1975: 6
1976: 6
1977: 6
1978: 6
1979: 6
1980: 6
1981: 6
1982: 6
1983: 6
1984: 6
1985: 6
1986: 6
1987: 6
1988: 6
1989: 6
1990: 6
1991: 6
1992: 6
1993: 6
1994: 6
1995: 6
1996: 6
1997: 6
1998: 6
1999: 6
2000: 6
2001: 6
2002: 6
2003: 6
2004: 6
2005: 6
2006: 6
2007: 6
2008: 6
2009: 6
2010: 6


3385: 8
3386: 8
3387: 8
3388: 8
3389: 8
3390: 8
3391: 8
3392: 8
3393: 8
3394: 8
3395: 8
3396: 8
3397: 8
3398: 8
3399: 8
3400: 8
3401: 8
3402: 8
3403: 8
3404: 8
3405: 8
3406: 8
3407: 8
3408: 8
3409: 8
3410: 8
3411: 8
3412: 8
3413: 8
3414: 8
3415: 8
3416: 8
3417: 8
3418: 8
3419: 8
3420: 8
3421: 8
3422: 8
3423: 8
3424: 8
3425: 8
3426: 8
3427: 8
3428: 8
3429: 8
3430: 8
3431: 8
3432: 8
3433: 8
3434: 8
3435: 8
3436: 8
3437: 8
3438: 8
3439: 8
3440: 8
3441: 8
3442: 8
3443: 8
3444: 8
3445: 8
3446: 8
3447: 8
3448: 8
3449: 8
3450: 8
3451: 8
3452: 8
3453: 8
3454: 8
3455: 8
3456: 8
3457: 8
3458: 8
3459: 8
3460: 8
3461: 8
3462: 8
3463: 8
3464: 8
3465: 8
3466: 8
3467: 8
3468: 8
3469: 8
3470: 8
3471: 8
3472: 8
3473: 8
3474: 8
3475: 8
3476: 8
3477: 8
3478: 8
3479: 8
3480: 8
3481: 8
3482: 8
3483: 8
3484: 8
3485: 8
3486: 8
3487: 8
3488: 8
3489: 8
3490: 8
3491: 8
3492: 8
3493: 8
3494: 8
3495: 8
3496: 8
3497: 8
3498: 8
3499: 8
3500: 8
3501: 8
3502: 8
3503: 8
3504: 8
3505: 8
3506: 8
3507: 8
3508: 8
3509: 8


5251: 10
5252: 10
5253: 10
5254: 10
5255: 10
5256: 10
5257: 10
5258: 10
5259: 10
5260: 10
5261: 10
5262: 10
5263: 10
5264: 10
5265: 10
5266: 10
5267: 10
5268: 10
5269: 10
5270: 10
5271: 10
5272: 10
5273: 10
5274: 10
5275: 10
5276: 10
5277: 10
5278: 10
5279: 10
5280: 10
5281: 10
5282: 10
5283: 10
5284: 10
5285: 10
5286: 10
5287: 10
5288: 10
5289: 10
5290: 10
5291: 10
5292: 10
5293: 10
5294: 10
5295: 10
5296: 10
5297: 10
5298: 10
5299: 10
5300: 10
5301: 10
5302: 10
5303: 10
5304: 10
5305: 10
5306: 10
5307: 10
5308: 10
5309: 10
5310: 10
5311: 10
5312: 10
5313: 10
5314: 10
5315: 10
5316: 10
5317: 10
5318: 10
5319: 10
5320: 10
5321: 10
5322: 10
5323: 10
5324: 10
5325: 10
5326: 10
5327: 10
5328: 10
5329: 10
5330: 10
5331: 10
5332: 10
5333: 10
5334: 10
5335: 10
5336: 10
5337: 10
5338: 10
5339: 10
5340: 10
5341: 10
5342: 10
5343: 10
5344: 10
5345: 10
5346: 10
5347: 10
5348: 10
5349: 10
5350: 10
5351: 10
5352: 10
5353: 10
5354: 10
5355: 10
5356: 10
5357: 10
5358: 10
5359: 10
5360: 10
5361: 10
5

6884: 12
6885: 12
6886: 12
6887: 12
6888: 12
6889: 12
6890: 12
6891: 12
6892: 12
6893: 12
6894: 12
6895: 12
6896: 12
6897: 12
6898: 12
6899: 12
6900: 12
6901: 12
6902: 12
6903: 12
6904: 12
6905: 12
6906: 12
6907: 12
6908: 12
6909: 12
6910: 12
6911: 12
6912: 12
6913: 12
6914: 12
6915: 12
6916: 12
6917: 12
6918: 12
6919: 12
6920: 12
6921: 12
6922: 12
6923: 12
6924: 12
6925: 12
6926: 12
6927: 12
6928: 12
6929: 12
6930: 12
6931: 12
6932: 12
6933: 12
6934: 12
6935: 12
6936: 12
6937: 12
6938: 12
6939: 12
6940: 12
6941: 12
6942: 12
6943: 12
6944: 12
6945: 12
6946: 12
6947: 12
6948: 12
6949: 12
6950: 12
6951: 12
6952: 12
6953: 12
6954: 12
6955: 12
6956: 12
6957: 12
6958: 12
6959: 12
6960: 12
6961: 12
6962: 12
6963: 12
6964: 12
6965: 12
6966: 12
6967: 12
6968: 12
6969: 12
6970: 12
6971: 12
6972: 12
6973: 12
6974: 12
6975: 12
6976: 12
6977: 12
6978: 12
6979: 12
6980: 12
6981: 12
6982: 12
6983: 12
6984: 12
6985: 12
6986: 12
6987: 12
6988: 12
6989: 12
6990: 12
6991: 12
6992: 12
6993: 12
6994: 12
6

8884: 13
8885: 13
8886: 13
8887: 13
8888: 13
8889: 13
8890: 13
8891: 13
8892: 13
8893: 13
8894: 13
8895: 13
8896: 13
8897: 13
8898: 13
8899: 13
8900: 13
8901: 13
8902: 13
8903: 13
8904: 13
8905: 13
8906: 13
8907: 13
8908: 13
8909: 13
8910: 13
8911: 13
8912: 13
8913: 13
8914: 13
8915: 13
8916: 13
8917: 13
8918: 13
8919: 13
8920: 13
8921: 13
8922: 13
8923: 13
8924: 13
8925: 13
8926: 13
8927: 13
8928: 13
8929: 13
8930: 13
8931: 13
8932: 13
8933: 13
8934: 13
8935: 13
8936: 13
8937: 13
8938: 13
8939: 13
8940: 13
8941: 13
8942: 13
8943: 13
8944: 13
8945: 13
8946: 13
8947: 13
8948: 13
8949: 13
8950: 13
8951: 13
8952: 13
8953: 13
8954: 13
8955: 13
8956: 13
8957: 13
8958: 13
8959: 13
8960: 13
8961: 13
8962: 13
8963: 13
8964: 13
8965: 13
8966: 13
8967: 13
8968: 13
8969: 13
8970: 13
8971: 13
8972: 13
8973: 13
8974: 13
8975: 13
8976: 13
8977: 13
8978: 13
8979: 13
8980: 13
8981: 13
8982: 13
8983: 13
8984: 13
8985: 13
8986: 13
8987: 13
8988: 13
8989: 13
8990: 13
8991: 13
8992: 13
8993: 13
8994: 13
8

10806: 15
10807: 15
10808: 15
10809: 15
10810: 15
10811: 15
10812: 15
10813: 15
10814: 15
10815: 15
10816: 15
10817: 15
10818: 15
10819: 15
10820: 15
10821: 15
10822: 15
10823: 15
10824: 15
10825: 15
10826: 15
10827: 15
10828: 15
10829: 15
10830: 15
10831: 15
10832: 15
10833: 15
10834: 15
10835: 15
10836: 15
10837: 15
10838: 15
10839: 15
10840: 15
10841: 15
10842: 15
10843: 15
10844: 15
10845: 15
10846: 15
10847: 15
10848: 15
10849: 15
10850: 15
10851: 15
10852: 15
10853: 15
10854: 15
10855: 15
10856: 15
10857: 15
10858: 15
10859: 15
10860: 15
10861: 15
10862: 15
10863: 15
10864: 15
10865: 15
10866: 15
10867: 15
10868: 15
10869: 15
10870: 15
10871: 15
10872: 15
10873: 15
10874: 15
10875: 15
10876: 15
10877: 15
10878: 15
10879: 15
10880: 15
10881: 15
10882: 15
10883: 15
10884: 15
10885: 15
10886: 15
10887: 15
10888: 15
10889: 15
10890: 15
10891: 15
10892: 15
10893: 15
10894: 15
10895: 15
10896: 15
10897: 15
10898: 15
10899: 15
10900: 15
10901: 15
10902: 15
10903: 15
10904: 15
10905: 15


In [44]:
timing = {}
ms = 0
infections = -1

for time in result:
    

    if time[0] != ms:
        infections += 1
        timing[infections] = {'demoNotes': [time[1], time[2], time[3]]}
        timing[infections]['delay'] = time[0] - ms
        ms = time[0]

    else:
        for i in range(1,4):
            timing[infections]['demoNotes'].append(time[i])

            
            
            
#     if time[0] != ms:
#         timing[time[0]] = {'demoNotes': [time[1], time[2], time[3]]}
#         timing[time[0]]['delay'] = time[0] - ms
#         ms = time[0]

#     else:
#         for i in range(1,4):
#             timing[time[0]]['demoNotes'].append(time[i])


In [45]:
timing



{0: {'demoNotes': [5, 78, 0], 'delay': 55333.333333333336},
 1: {'demoNotes': [1, 80, 0], 'delay': 1333.3333333333358},
 2: {'demoNotes': [5, 63, 0], 'delay': 20000.0},
 3: {'demoNotes': [3, 55, 0], 'delay': 4666.6666666666715},
 4: {'demoNotes': [4, 58, 0], 'delay': 7333.3333333333285},
 5: {'demoNotes': [3, 65, 0], 'delay': 666.6666666666715},
 6: {'demoNotes': [5, 68, 0], 'delay': 2000.0},
 7: {'demoNotes': [5, 48, 0, 5, 68, 0], 'delay': 4000.0},
 8: {'demoNotes': [0, 68, 0], 'delay': 4666.6666666666715},
 9: {'demoNotes': [5, 67, 0], 'delay': 2666.666666666657},
 10: {'demoNotes': [1, 62, 0], 'delay': 1333.3333333333285},
 11: {'demoNotes': [2, 67, 0], 'delay': 1.4551915228366852e-11},
 12: {'demoNotes': [1, 74, 0], 'delay': 2000.0},
 13: {'demoNotes': [1, 55, 0], 'delay': 1333.3333333333285},
 14: {'demoNotes': [0, 51, 0], 'delay': 4000.0},
 15: {'demoNotes': [2, 51, 0], 'delay': 2666.6666666666715},
 16: {'demoNotes': [5, 74, 0], 'delay': 666.666666666657},
 17: {'demoNotes': [5,